In [1]:
import pandas as pd
import numpy as np

import torch

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
data_path = "./Refined_Data/Grouped_Data/Input_Data2.csv"

building_num = [[34]]

repeat = 10

In [3]:
data = pd.read_csv(data_path, parse_dates = ["date_time"])

data['num']     =   data['num'].apply(str)
data['Week']    =   data['Week'].apply(str)
data['24Hour']  =   data['24Hour'].apply(str)
data['holiday'] =   data['holiday'].apply(str)
data['Weekend'] =   data['Weekend'].apply(str)
data['energy_group'] = data['energy_group'].apply(str)

In [4]:
data_bag = [data.loc[data["num"] == str(building_num[i])].copy() for i in range(len(building_num))]

data_bag = []
for buildings in building_num:
    temp = data.loc[data["num"] == str(buildings[0])].copy()
    for num_idx in range(1, len(buildings)):
        temp = pd.concat([temp, data.loc[data["num"] == str(buildings[num_idx])].copy()])
    data_bag.append(temp)

In [5]:
for ___ in range(repeat):
    for idx in range(len(data_bag)):

        data = data_bag[idx]

        torch.cuda.empty_cache()

        max_prediction_length = 24
        max_encoder_length = 168
        training_cutoff = data["time_idx"].max() - max_prediction_length

        training = TimeSeriesDataSet(
            data[lambda x: x.time_idx <= training_cutoff],
            time_idx="time_idx",
            target="kWH",
            group_ids=["num"],
            min_encoder_length=max_encoder_length//2,
            max_encoder_length=max_encoder_length,
            min_prediction_length=3,
            max_prediction_length=max_prediction_length,
            static_categoricals=["num", "energy_group"],
            static_reals=["non_electric_aircondition", "sunlight"],
            time_varying_known_categoricals=["Week", "24Hour", "holiday", "Weekend"],
            time_varying_known_reals=["C", "m/s", "wet", "mm", "hr", "time_idx", "perceived_temperature", "discomfort_index"],
            time_varying_unknown_categoricals=[],
            time_varying_unknown_reals=["kWH"],
            add_relative_time_idx=True,
            add_target_scales=True,
            add_encoder_length=True,
        )

        # create validation set (predict=True) which means to predict the last max_prediction_length points in time
        # for each series
        validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

        # create dataloaders for model
        batch_size = 64  # set this between 32 to 128
        train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
        val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

        # configure network and trainer
        early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
        lr_logger = LearningRateMonitor()  # log the learning rate
        logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

        trainer = pl.Trainer(
            max_epochs=500,
            gpus=1,
            weights_summary="top",
            gradient_clip_val=0.14,
            limit_train_batches=30, 
            callbacks=[lr_logger, early_stop_callback],
            logger=logger,
        )


        tft = TemporalFusionTransformer.from_dataset(
            training,
            learning_rate=0.03,
            hidden_size=64,
            lstm_layers = 2,
            attention_head_size=4,
            dropout=0.15,
            hidden_continuous_size=8,
            output_size=7,
            loss=QuantileLoss(),
            log_interval=0,
            reduce_on_plateau_patience=4,
        )

        # fit network
        trainer.fit(
            tft,
            train_dataloader=train_dataloader,
            val_dataloaders=val_dataloader,
        )

rch1/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/joeunchan/anaconda3/envs/torch1/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
Epoch 0:  97%|█████████▋| 30/31 [00:08<00:00,  3.41it/s, loss=71.6, v_num=7, val_loss=176.0, train_loss_step=57.30]
Validating: 0it [00:00, ?it/s]
Epoch 1:  97%|█████████▋| 30/31 [00:08<00:00,  3.56it/s, loss=51.9